### Benson Team Awesome

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


|Code      | explanation                                                                                     |
|:-------- |:----------------------------------------------------------------------------------------------- | 
| C/A      | Control Area (A002)                                                                          | 
| UNIT     | Remote Unit for a station (R051)                                                             | 
| SCP      | Subunit Channel Position represents an specific address for a device (02-00-00)              | 
| STATION  | Represents the station name the device is located at                                         | 
| LINENAME | Represents all train lines that can be boarded at this station                               | 
|          |   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for | 
|          |   4, 5, 6, N, Q, and R trains.                                                                  | 
| DIVISION | Represents the Line originally the station belonged to BMT, IRT, or IND                      | 
| DATE     | Represents the date (MM-DD-YY)                                                               |   
| TIME     | Represents the time (hh:mm:ss) for a scheduled audit event                                   | 
| DESc     | Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)                | 
|          |   1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.         | 
|          |   2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. |
|ENTRIES   | The comulative entry register value for a device|
|EXIST     | The cumulative exit register value for a device|


In [3]:
import csv
import urllib
import io
import pandas as pd
import numpy as np
import datetime as dt
import multiprocessing as mlp
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt')

In [13]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,id
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027,"A002,R051,02-00-00,59 ST"
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032,"A002,R051,02-00-00,59 ST"
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059,"A002,R051,02-00-00,59 ST"
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162,"A002,R051,02-00-00,59 ST"
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248,"A002,R051,02-00-00,59 ST"


In [14]:
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'id'],
      dtype='object')

In [15]:
data.rename(columns={data.columns[-1]: data.columns[-1].strip()}, inplace=True)
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'id'],
      dtype='object')

In [16]:
#data[mask, 'Delta'] = abs((data[mask,'ENTRIES'] - data[mask,'ENTRIES'].shift(-1)) + (data[mask,'EXITS'] - data[mask,'EXITS'].shift(-1)))
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,id
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027,"A002,R051,02-00-00,59 ST"
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032,"A002,R051,02-00-00,59 ST"
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059,"A002,R051,02-00-00,59 ST"
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162,"A002,R051,02-00-00,59 ST"
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248,"A002,R051,02-00-00,59 ST"


In [30]:
#test = data.sort_values(['Delta'])
data.drop(['C/A', 'UNIT', 'SCP','STATION'], axis=1, inplace=True)

ValueError: labels ['C/A' 'UNIT' 'SCP' 'STATION'] not contained in axis

In [32]:
data['Period'] = data['DATE'] + ' ' + data['TIME']
data.Period = pd.to_datetime(data.Period)
data.drop(['DATA','TIME'])

ValueError: labels ['DATA' 'TIME'] not contained in axis

In [31]:
#df_night = data[data['TIME' ==  ]
data.head()

,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,id,Period
0,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027,"A002,R051,02-00-00,59 ST",2016-03-19 00:00:00
1,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032,"A002,R051,02-00-00,59 ST",2016-03-19 04:00:00
2,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059,"A002,R051,02-00-00,59 ST",2016-03-19 08:00:00
3,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162,"A002,R051,02-00-00,59 ST",2016-03-19 12:00:00
4,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248,"A002,R051,02-00-00,59 ST",2016-03-19 16:00:00


In [18]:
#Create the keys that is used to make a mask

data['id'] = data.iloc[:, 0]
for j in range(1,4):
        data.ix[:,'id'] += ',' + data.iloc[:,j]
        
#drop the keys that is 
mask_key = data['id'].drop_duplicates()
print(data.head())

    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  00:00:00   
1  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  04:00:00   
2  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  08:00:00   
3  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  12:00:00   
4  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  16:00:00   

      DESC  ENTRIES    EXITS                        id  
0  REGULAR  5590801  1889027  A002,R051,02-00-00,59 ST  
1  REGULAR  5590828  1889032  A002,R051,02-00-00,59 ST  
2  REGULAR  5590852  1889059  A002,R051,02-00-00,59 ST  
3  REGULAR  5590958  1889162  A002,R051,02-00-00,59 ST  
4  REGULAR  5591207  1889248  A002,R051,02-00-00,59 ST  


In [19]:
#Initialize the Delta columns as zeros
data['Delta'] = np.zeros(data.shape[0])

#temporary function to fill in the deltas by selections
def fill_in(mask):
    data.loc[mask,'Delta'] = (data.loc[mask, 'ENTRIES'] - data.loc[mask, 'ENTRIES'].shift(-1))

In [20]:
for s in mask_key:
    mask = data.ix[:,'id'] == s
    fill_in(mask)

In [22]:
#print(data.head(20))
#print(data.tail(50))
mask

          C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
192896  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/24/2016   
192897  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/24/2016   
192898  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192899  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192900  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192901  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192902  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192903  TRAM2  R469  00-05-00  RIT-ROOSEVELT        R      RIT  03/25/2016   
192904  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  03/19/2016   
192905  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  03/19/2016   
192906  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  03/19/2016   
192907  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT  